<a href="https://colab.research.google.com/github/alyssonrafael/Algoritimos_e_complexidade/blob/main/desafio3/desafio_3_raspagem_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio 3 WebScraping

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# URL do portal de notícias
URL = "https://jc.uol.com.br/"


# --- FUNÇÃO PARA EXTRAIR A DATA DE UMA PÁGINA ---
def obter_data_publicacao(url_noticia):
    """Acessa a URL de uma notícia e extrai sua data de publicação."""
    try:
        pagina_noticia = requests.get(url_noticia, timeout=10)
        pagina_noticia.raise_for_status()
        sopa_noticia = BeautifulSoup(pagina_noticia.content, 'html.parser')

        info_publicacao = sopa_noticia.find('small', class_='publishinfo')

        if info_publicacao:
            texto_completo = info_publicacao.get_text(strip=True)
            return texto_completo
        else:
            return 'Data não encontrada na página'

    except requests.exceptions.RequestException as e:
        print(f"  (Erro ao acessar link {url_noticia}: {e})")
        return 'Falha ao buscar data'

# 1. Acessando a página principal
try:
    site = requests.get(URL)
    site.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Erro ao acessar a página principal: {e}")
    exit()

# 2. Raspando os dados
soup = BeautifulSoup(site.content, 'html.parser')
noticias_lista = []

print("--- INICIANDO EXTRAÇÃO DE MANCHETES E DATAS ---")

# 3. Identificando os elementos HTML (h1 e h4 com a classe 'title')
manchetes = soup.find_all(['h1', 'h4'], class_=re.compile(r'title'))

# 4. Capturando texto, link e data para cada manchete
for i, manchete in enumerate(manchetes):
    titulo = manchete.get_text(strip=True)
    link_tag = manchete.find_parent('a')
    link = link_tag['href'] if link_tag else None

    if titulo and link:
        print(f"\nProcessando notícia {i+1}/{len(manchetes)}: {titulo}")

        data_publicacao = obter_data_publicacao(link)

        tipo = "Manchete Principal" if manchete.name == 'h1' else "Manchete Secundária"

        noticia = {
            'tipo': tipo,
            'titulo': titulo,
            'link': link,
            'data': data_publicacao
        }
        noticias_lista.append(noticia)

        # 5. Exibindo os resultados no terminal
        print(f"  Tipo: {noticia['tipo']}")
        print(f"  Link: {noticia['link']}")
        print(f"  Data: {noticia['data']}")

        time.sleep(0.5)

# 6. Salvando em CSV
if noticias_lista:
    df = pd.DataFrame(noticias_lista)
    nome_arquivo = 'manchetes_jc_uol_final.csv'
    df = df[['tipo', 'titulo', 'link', 'data']]
    df.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')

    print(f"\n✔ Dados salvos com sucesso no arquivo '{nome_arquivo}'!")
else:
    print("\nNenhuma manchete foi encontrada com os critérios (h1 ou h4 com classe 'title').")

--- INICIANDO EXTRAÇÃO DE MANCHETES E DATAS ---

Processando notícia 1/57: Bolsonaro vai ser preso agora? Entenda o que acontece com os 8 condenados pelo STF
  Tipo: Manchete Principal
  Link: https://jc.uol.com.br/politica/2025/09/11/bolsonaro-vai-ser-preso-agora-entenda-o-que-acontece-apos-condenacao.html
  Data: Publicado em 11/09/2025 às 20:53

Processando notícia 2/57: Lula sobre julgamento da trama golpista: cada juiz vê aquilo que quer ver
  Tipo: Manchete Secundária
  Link: https://jc.uol.com.br/politica/2025/09/11/lula-sobre-julgamento-da-trama-golpista-cada-juiz-ve-aquilo-que-quer-ver.html
  Data: Publicado em 11/09/2025 às 20:48

Processando notícia 3/57: JC Educação: Saúde mental na escola é responsabilidade coletiva e precisa estar no centro do debate
  Tipo: Manchete Secundária
  Link: https://jc.uol.com.br/colunas/enem-e-educacao/2025/09/11/jc-educacao-saude-mental-na-escola-e-responsabilidade-coletiva-e-precisa-estar-no-centro-do-debate.html
  Data: Publicado em 11/09/2